In [1]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
from working_with_files import *
from preprocessing import *
import nltk
from datetime import datetime
from collections import Counter
from string import punctuation
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [2]:
# build vocabulary and train word2vec model
def build_w2v_model(corpus, min_count):
    documents = []
    for text_data in corpus:
        documents.extend(tokenize_text(text_data['lemmas']))
    w2v_model = gensim.models.Word2Vec(documents, size=300, window=5, min_count=min_count)
    return w2v_model

# tokenize text into list of lists for building w2v model
def tokenize_text(lemmas):
    stopwords = set(get_lexicon_from_file('ru_stopwords_extended.txt'))
    text = ' '.join(lemmas)
    sentences = sent_tokenize(text)
    sentences_tokenized = []
    for sentence in sentences:
        words = [token for token in word_tokenize(sentence) if token.isalpha() and token.isascii() == False 
                 and len(token) > 2 and token not in stopwords]
        if words != []:
            sentences_tokenized.append(words)
    return sentences_tokenized

# check if word in w2v-vocab
def check_if_word_in_vocab(model, word):
    return word in model.wv.vocab

# check how many input models vocabs contain input word 
def get_num_entries(word, models):
    num_entries = 0
    for model in models:
        if check_if_word_in_vocab(model, word):
            num_entries += 1
    return num_entries

# get jaccard similarity for two lists of words
def jaccard_similarity(list1, list2):
    s1, s2 = set(list1), set(list2)
    return len(s1 & s2) / len(s1 | s2)

# get w2v semantic associates
def get_most_similar_list(model, word, topn):
    return list(dict(model.wv.most_similar(positive=word, topn=topn)).keys())

# get semantic shift for every word
def detect_semantic_shifts(models, topn, num_top_shifts):
    # get words relative dfs to further exclude rare words from the result
    freq_dict = get_json_from_file('freq_dict.json')
    # join models vocabs
    joint_vocab = [] 
    for model in models:
        joint_vocab.extend(list(model.wv.vocab))
    # make new vocab containing only words appearing in both models vocabs
    vocab = [word for word in joint_vocab if get_num_entries(word, models) == len(models)]
    associates_dicts = []
    # get semantic associates for every model & every word in vocab
    for model in models:
        associates_dict = {}
        for word in vocab:
            if freq_dict[word] >= 0.005 and freq_dict[word] < 0.85:
                associates_dict[word] = get_most_similar_list(model, word, topn)
        associates_dicts.append(associates_dict)
        
    # get jaccard distance for every word in vocab
    jacc_similarities = {}
    if len(models) == 2:
        for word in associates_dicts[0].keys():
            jacc_similarity = jaccard_similarity(associates_dicts[0][word], associates_dicts[1][word])
            jacc_similarities[word] = jacc_similarity
        jacc_similarities_sorted = sorted(jacc_similarities.items(), key=lambda kv: kv[1])
        print(jacc_similarities_sorted[:num_top_shifts])
        return jacc_similarities_sorted
    else:
        indx_to_model = dict(enumerate(models))
        indx_pairs = list(nltk.bigrams(indx_to_model.keys()))
        for pair in indx_pairs:
            for word in associates_dicts[0].keys():
                jacc_similarity = jaccard_similarity(associates_dicts[pair[0]][word], associates_dicts[pair[1]][word])
                if word not in jacc_similarities:
                    jacc_similarities[word] = jacc_similarity    
                else:
                    jacc_similarities[word] += jacc_similarity  
        avg_jacc_similarities = {word: sim/(len(models)-1) for word, sim in jacc_similarities.items()}
        avg_jacc_similarities_sorted = sorted(avg_jacc_similarities.items(), key=lambda kv: kv[1])
        print(avg_jacc_similarities_sorted[:num_top_shifts])
        return avg_jacc_similarities_sorted
    
# get semantic associates for a word in a list of w2v models       
def get_most_similar(word, topn, models):
    print(word.upper(), '\n')
    titles = ['< 2008', '2008 - 2014', '> 2014']
    indx = 0
    for model in models:
        title = titles[indx]
        try:
            print(title, '\n', model.wv.most_similar(positive=word, topn=topn), '\n')
        except KeyError:
            print(title, '\nNo such word in vocabulary\n')
        indx += 1 

In [6]:
def sort_json_for_semantic_shifts(corpus):
    corpus1 = [] # < 2008
    corpus2 = [] # < 2014
    # corpus3 = corpus (all data)
    date1 = datetime.strptime('01-01-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('01-01-2014', '%d-%m-%Y').date()
    for text_data in corpus:
        if text_data['id'] not in [8079, 10995, 11494, 11848, 11854, 11858, 26987, 30920]:
            try:
                date = datetime.strptime(text_data['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(text_data['date'], '%m-%Y').date()
            if date <= date1:
                corpus1.append(text_data)
            if date <= date2:
                corpus2.append(text_data)
    return corpus1, corpus2, corpus

# count number of texts and words in a corpus
def get_json_stat(corpora, return_data=True):
    n = 0
    for corpus in corpora:
        num_words = 0
        for text_data in corpus:
            for pos_tag in text_data['pos_tags']:
                if pos_tag != 'NO_POS':
                    num_words += 1
        print('Corpus {}: {} texts, {} words'.format(n, len(corpus), num_words))
        n += 1
        if return_data:
            return len(corpus), num_words

In [4]:
corpus = get_json_from_file('mfa_texts_rhetoric.json')

In [10]:
subcorpora = sort_json_for_semantic_shifts(corpus)

In [15]:
for subcorpus in subcorpora:
    print(len(subcorpus))

7433
18309
34831


In [26]:
models = [build_w2v_model(subcorpus, 15) for subcorpus in subcorpora]

In [10]:
models = []
for n in range(3):
    models.append(KeyedVectors.load_word2vec_format('model' + str(n) + '.bin', binary=True))

In [11]:
len(models)

3

In [12]:
semantic_shifts = detect_semantic_shifts(models, 50, 100)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[('умеренный', 0.031037414965986394), ('образец', 0.03136629452418926), ('пропагандистский', 0.036297250859106525), ('выявлять', 0.03651987110633727), ('избиратель', 0.03651987110633727), ('революция', 0.03696539866752632), ('предвыборный', 0.03763440860215054), ('крым', 0.041666666666666664), ('постановка', 0.04737881114279447), ('возвращаться', 0.0478384902348036), ('правый', 0.0478384902348036), ('добро', 0.0478384902348036), ('журналистский', 0.04945054945054945), ('видео', 0.05263157894736842), ('убедительный', 0.05274822695035461), ('похоже', 0.053098326127923735), ('остальное', 0.053098326127923735), ('периодически', 0.053682342502218275), ('держать', 0.0545010545010545), ('грань', 0.0545010545010545), ('эпизод', 0.0545010545010545), ('вспоминать', 0.05823068309070548), ('секрет', 0.05894217839533841), ('обходиться', 0.05894217839533841), ('интересно', 0.0606060606060606), ('офис', 0.06382978723404255), ('пресловутый', 0.06382978723404255), ('отдельно', 0.06395019807583474), ('з

In [14]:
# print top 100
for pair in semantic_shifts[:100]:
    print(pair)

('умеренный', 0.031037414965986394)
('образец', 0.03136629452418926)
('пропагандистский', 0.036297250859106525)
('выявлять', 0.03651987110633727)
('избиратель', 0.03651987110633727)
('революция', 0.03696539866752632)
('предвыборный', 0.03763440860215054)
('крым', 0.041666666666666664)
('постановка', 0.04737881114279447)
('возвращаться', 0.0478384902348036)
('правый', 0.0478384902348036)
('добро', 0.0478384902348036)
('журналистский', 0.04945054945054945)
('видео', 0.05263157894736842)
('убедительный', 0.05274822695035461)
('похоже', 0.053098326127923735)
('остальное', 0.053098326127923735)
('периодически', 0.053682342502218275)
('держать', 0.0545010545010545)
('грань', 0.0545010545010545)
('эпизод', 0.0545010545010545)
('вспоминать', 0.05823068309070548)
('секрет', 0.05894217839533841)
('обходиться', 0.05894217839533841)
('интересно', 0.0606060606060606)
('офис', 0.06382978723404255)
('пресловутый', 0.06382978723404255)
('отдельно', 0.06395019807583474)
('знакомый', 0.06431159420289854

In [16]:
# semantic associates for every word on top
for pair in semantic_shifts[:100]:
    print(get_most_similar(pair[0], 50, models))

УМЕРЕННЫЙ 

< 2008 
 [('сбалансировать', 0.8971379995346069), ('макроэкономический', 0.8959012031555176), ('прагматичный', 0.8941746950149536), ('подчинять', 0.8872907161712646), ('случайный', 0.8674476146697998), ('неприсоединение', 0.8628067970275879), ('инстинкт', 0.8624404668807983), ('снижаться', 0.8608394861221313), ('чужой', 0.859222412109375), ('прозрачный', 0.85832279920578), ('кардинальный', 0.8578999042510986), ('последовательность', 0.8564753532409668), ('доминировать', 0.8564468622207642), ('бояться', 0.8536624908447266), ('отрицать', 0.8523584604263306), ('усматривать', 0.8512517809867859), ('многовекторный', 0.850967526435852), ('сходный', 0.8499397039413452), ('ошибаться', 0.8481508493423462), ('одинаковый', 0.847969651222229), ('конфронтационный', 0.8451398611068726), ('неправильно', 0.8445760011672974), ('склонный', 0.8408839702606201), ('помешать', 0.8403552770614624), ('многовекторность', 0.8393348455429077), ('зрелость', 0.8390539884567261), ('мягко', 0.83858692646

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


2008 - 2014 
 [('читать', 0.8665887117385864), ('прочитывать', 0.8290696740150452), ('комментировать', 0.8003922700881958), ('новость', 0.7896760702133179), ('приводиться', 0.7821420431137085), ('спецкор', 0.7750155329704285), ('радиостанция', 0.7741408944129944), ('катерина', 0.7709417343139648), ('правозащитник', 0.7679182887077332), ('поинтересоваться', 0.7664234638214111), ('газета', 0.7661086916923523), ('эхо', 0.7625809907913208), ('узнавать', 0.7584750652313232), ('публиковаться', 0.7564873695373535), ('опубликовывать', 0.7526621222496033), ('интервью', 0.7499161958694458), ('редкий', 0.7492601871490479), ('преподносить', 0.7486950159072876), ('выпускать', 0.7464048862457275), ('борис', 0.7440561056137085), ('тенденциозный', 0.7428795099258423), ('изучаться', 0.7421189546585083), ('уезжать', 0.7412694692611694), ('освещаться', 0.7408671379089355), ('виктор', 0.7392281889915466), ('вспоминать', 0.7387834787368774), ('риа', 0.7360877394676208), ('александрович', 0.7358946204185486

> 2014 
 [('исчерпывать', 0.6885180473327637), ('раскрываться', 0.6530017852783203), ('оперировать', 0.6450254917144775), ('проливать', 0.6095664501190186), ('выкладывать', 0.5950456261634827), ('доносить', 0.5939496755599976), ('полно', 0.5930184721946716), ('пригождаться', 0.5911082029342651), ('заглядывать', 0.5862448811531067), ('точный', 0.5841721892356873), ('перепроверять', 0.5810983777046204), ('делиться', 0.5745195746421814), ('приводиться', 0.5729867219924927), ('уяснять', 0.5725598931312561), ('упускать', 0.5711157321929932), ('сокрытие', 0.5706307291984558), ('доступный', 0.5682716369628906), ('взглядывать', 0.566609799861908), ('публиковаться', 0.5660579800605774), ('владеть', 0.5654329657554626), ('набираться', 0.5653873682022095), ('аккумулировать', 0.5604719519615173), ('публиковать', 0.5602047443389893), ('проверять', 0.558506965637207), ('выяснивать', 0.5575706958770752), ('узнавать', 0.556759238243103), ('утаивать', 0.5539530515670776), ('формулирование', 0.550575017

2008 - 2014 
 [('предположение', 0.6936158537864685), ('картина', 0.6830629110336304), ('представление', 0.6803327798843384), ('цитата', 0.6717024445533752), ('суждение', 0.6619959473609924), ('эмоционально', 0.6493052244186401), ('термин', 0.6461142301559448), ('версия', 0.6460205912590027), ('аналитик', 0.6449105739593506), ('выкладка', 0.6389576196670532), ('излишний', 0.6379159688949585), ('клише', 0.6326571106910706), ('нечего', 0.6304771900177002), ('намеренно', 0.6303846836090088), ('замечание', 0.6301052570343018), ('характеристика', 0.62912517786026), ('непредвзято', 0.6275439262390137), ('картинка', 0.6266993284225464), ('извинять', 0.6260439157485962), ('негатив', 0.6242263317108154), ('пища', 0.6229294538497925), ('искажение', 0.6181491613388062), ('довод', 0.617655873298645), ('тенденциозный', 0.6134929656982422), ('домысел', 0.6118305921554565), ('корректно', 0.6086856126785278), ('интерпретация', 0.6072194576263428), ('мода', 0.6071208715438843), ('грешить', 0.6064242124

In [49]:
get_texts_by_lemmas(['образец'], subcorpora[1], [subcorpora[0]])

31-12-2008 28 декабря произошло историческое событие: России возвращено Сергиевское подворье в Иерусалиме - символ российского присутствия в Святой Земле. Завершена перерегистрация на Российскую Федерацию уникального исторического достояния, созданного в конце XIX века силами и на средства Императорского Православного Палестинского общества (ИППО).
Как известно, в 2005 году после визита Президента Российской Федерации на Святую Землю было дано поручение в кратчайшие сроки обеспечить восстановление российских прав собственности на эту дорогую сердцу россиян реликвию. В течение последних лет Министерство иностранных дел и Посольство России в Израиле при активном участии ИППО вели интенсивные переговоры с израильской стороной. Эти усилия осуществлялись под постоянным контролем и при поддержке Президента Российской Федерации Д.А.Медведева.
Сергиевское подворье является редким по красоте и величию образцом архитектуры XIX века, украшено уникальными фресками русских мастеров. Сегодня этот ко

24-05-2013  Вопрос серьезный. Вы сами видите, насколько активно действует международная террористическая сеть. Это подтвердил и теракт в Бостоне. Вам известно, какие активные операции по нейтрализации террористов проводятся в целом ряде регионов Северного Кавказа. Национальный антитеррористический комитет (НАК) действует открыто с точки зрения предоставления материалов для СМИ. Недавно НАК провел в Пресс-центре МИД мероприятие для прессы, где были показаны образцы снаряжения и техники, изощренно используемых террористическими группами в России.
Это один из опаснейших вызовов общей безопасности, с которым нужно бороться коллективными усилиями. На эту тему вчера была интересная дискуссия на Московской международной конференции по вопросам европейской безопасности». Все участники подтверждали, что одна из самых важных и серьезных современных угроз - международный терроризм. 


11-06-2009  Посольством поддерживаются постоянные контакты с местными властями для решения всех необходимых вопро